In [ ]:
import os
import sys
import torch
import cv2
import random
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
from PIL import Image
from tqdm import tqdm
from pathlib import Path
from torch.utils.data import DataLoader

WORK_DIR = Path(Path.cwd()).parent
sys.path.append(str(WORK_DIR))
from src.datasets import get_dataset, get_dataloader
from src.utils import parse_data_cfg, IMG, FPHA, LMDB, DATA_DIR, YOLO

In [ ]:
cfgname = 'debug/yolov2_fpha_hpo_bbox_2hand_1_data_OVERFIT'
cfg = parse_data_cfg(WORK_DIR/'data_cfg'/(cfgname + '.cfg'))
epoch = 1000
exp_dir = cfg["exp_dir"]
data_split = 'train'
split_set = cfg[data_split + '_set']

# Visualize Dataloader

In [ ]:
def collate_fn(batch):
    """
    Flip entire dataset batch to same side
    Args:
        batch   : list of img, bbox_gt, uvd_gt
        img     : [img_1, ..., img_batch]
        bbox_gt : [bbox_gt_1, ..., bbox_gt_batch]
        uvd_gt  : [uvd_gt_1, ..., uvd_gt_batch]
    Out:
        Vertically mirrored inputs
    """
    FT                      = torch.FloatTensor
    img, bbox_gt, uvd_gt    = zip(*batch)
    flip                    = random.randint(1, 10000)%2
    # Do flipping
    # 0 = left, 1 = right
    hand_side = 1
    if flip:
        hand_side = 0  

    new_img     = []
    new_bbox    = []
    new_uvd     = []
    for i, b, u in batch:
        if flip:
            i       = i.transpose(Image.FLIP_LEFT_RIGHT)
            b[0]    = 0.999 - b[0]
            u[:, 0] = 0.999 - u[:, 0]
        i = np.asarray(i)
        i = i/255.0
        i = IMG.imgshape2torch(i)
        new_img.append(i)
        new_bbox.append(b)
        new_uvd.append(u)

    new_img     = FT(new_img)
    new_bbox    = FT(new_bbox)
    new_uvd     = FT(new_uvd)
    return new_img, new_bbox, new_uvd, hand_side

In [ ]:
dataset_kwargs = {'split_set': split_set}
dataset   = get_dataset(cfg, dataset_kwargs)
sampler   = None
shuffle   = cfg['shuffle']
kwargs = {'batch_size'  : int(cfg['batch_size']),
          'shuffle'     : shuffle,
          'num_workers' : int(cfg['num_workers']),
          'pin_memory'  : True,
          'collate_fn'  : collate_fn}
data_loader = get_dataloader(dataset, sampler, kwargs)

In [ ]:
idx = 0 
for i, (img, bbox_gt, uvd_gt, hand_side) in enumerate(data_loader):
    if i == idx:
        if hand_side == 0:
            print('left')
        else:
            print('right')
        batch_size = img.shape[0]
        img = img.cpu().numpy()
        img = np.swapaxes(img, 2, 3)
        img = np.swapaxes(img, 1, 3)
        img = IMG.scale_img_255(img)
        bbox_gt = bbox_gt.cpu().numpy()
        uvd_gt = uvd_gt.cpu().numpy()
        uvd_gt = np.squeeze(uvd_gt)
        break
    i += 1

In [ ]:
fig, ax = plt.subplots(4, 4, figsize=(15, 15))
idx = 0
for i in range(4):
    for j in range(4):
        k = np.ravel_multi_index((i, j), (4, 4))
        if k >= len(img):
            break
        cur_img = img[idx]
        b = bbox_gt[idx]
        u = uvd_gt[idx]        
        ax[i, j].imshow(cur_img)
        FPHA.draw_bbox(ax[i, j], b, (cur_img.shape[1], cur_img.shape[0]))
        u = IMG.scale_points_WH(u, (1,1), (cur_img.shape[0], cur_img.shape[1]))
        FPHA.visualize_joints_2d(ax[i, j], u[FPHA.REORDER_IDX], joint_idxs=False)
        idx += 1

# Evaluation

In [ ]:
keys = LMDB.get_keys(os.path.join(DATA_DIR, split_set + "_keys_cache.p"))
bbox_gt = LMDB.read_all_lmdb_dataroot(keys, os.path.join(DATA_DIR, split_set + "_bbox_gt.lmdb"), "float32", 4)
pred_file = os.path.join(DATA_DIR, exp_dir, 'predict_{}_{}_bbox.txt'.format(epoch, data_split))
bbox_pred = np.loadtxt(pred_file)
bbox_pred = np.asarray([bbox[:5] for bbox in bbox_pred]) # take only the first bbox

xyz_gt = LMDB.read_all_lmdb_dataroot(keys, os.path.join(DATA_DIR, split_set + '_xyz_gt.lmdb'), 'float32', (21, 3))
uvd_gt = FPHA.xyz2uvd_color(xyz_gt)

# Left hand
pred_file = os.path.join(DATA_DIR, exp_dir, 'predict_{}_{}_uvd_left.txt'.format(epoch, data_split))
pred_uvd_left = np.reshape(np.loadtxt(pred_file), (-1, 21, 3))
pred_uvd_left = IMG.scale_points_WH(pred_uvd_left, (1, 1), (1920, 1080))
pred_uvd_left[..., 2] *= 1000
pred_xyz_left = FPHA.uvd2xyz_color(pred_uvd_left)

pred_file = os.path.join(DATA_DIR, exp_dir, 'predict_{}_{}_conf_left.txt'.format(epoch, data_split))
pred_conf_left = np.loadtxt(pred_file)

# Right hand
pred_file = os.path.join(DATA_DIR, exp_dir, 'predict_{}_{}_uvd_right.txt'.format(epoch, data_split))
pred_uvd_right = np.reshape(np.loadtxt(pred_file), (-1, 21, 3))
pred_uvd_right = IMG.scale_points_WH(pred_uvd_right, (1, 1), (1920, 1080))
pred_uvd_right[..., 2] *= 1000
pred_xyz_right = FPHA.uvd2xyz_color(pred_uvd_right)

pred_file = os.path.join(DATA_DIR, exp_dir, 'predict_{}_{}_conf_right.txt'.format(epoch, data_split))
pred_conf_right = np.loadtxt(pred_file)

In [ ]:
idx = 1
print(keys[idx])

img = Image.open(os.path.join(DATA_DIR, 'First_Person_Action_Benchmark', 'Video_files', keys[idx]))
img = img.transpose(Image.FLIP_LEFT_RIGHT)
img = np.asarray(img)
fig, ax = plt.subplots()
ax.imshow(img)

FPHA.draw_bbox(ax, bbox_pred[idx], (img.shape[1], img.shape[0]), 'r')
FPHA.visualize_joints_2d(ax, pred_uvd_left[idx][FPHA.REORDER_IDX], joint_idxs=False, c='r')
FPHA.visualize_joints_2d(ax, pred_uvd_right[idx][FPHA.REORDER_IDX], joint_idxs=False, c='g')
FPHA.draw_bbox(ax, bbox_gt[idx], (img.shape[1], img.shape[0]), 'b')
FPHA.visualize_joints_2d(ax, uvd_gt[idx][FPHA.REORDER_IDX], joint_idxs=False, c='b')